In [1]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())
print(torch.version.cuda)
print('Location:', torch.__file__) # /home/emre/.local/lib/python3.7/site-packages

1.12.0+cu102
True
10.2
Location: /home/emre/anaconda3/envs/emre_venv/lib/python3.9/site-packages/torch/__init__.py


/home/emre/anaconda3/envs/emre_venv/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from typing import Union, Tuple
from torch_geometric.typing import OptPairTensor, Adj, Size # Optional[Tensor], Union[Tensor, SparseTensor], Optional[Tuple[int, int]], all about data type

from torch import Tensor
from torch.nn import Linear
import torch.nn.functional as F
from torch_sparse import SparseTensor, matmul
from torch_geometric.nn.conv import MessagePassing

In [3]:
import warnings
warnings.filterwarnings("ignore")
from joblib import Parallel, delayed

from sklearn.preprocessing import StandardScaler, MinMaxScaler

import torch.nn as nn
import torch.nn.functional as F
import torch_geometric.nn as gnn
from torchsummary import summary
import torchvision
from torch_geometric.nn import MessagePassing
from torch_geometric.data import Dataset, Data, DataLoader
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp
from torch_geometric.utils import softmax

In [4]:
import pickle
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output
!CUDA_LAUNCH_BLOCKING=0
torch.cuda.set_device(2)
print('Num GPU:', torch.cuda.device_count())
print('Current device ID:', torch.cuda.current_device())
print('Device name:', torch.cuda.get_device_name())

Num GPU: 3
Current device ID: 2
Device name: TITAN V


In [5]:
def load_pkl(filename):
    with open(filename, 'rb') as input:
        data = pickle.load(input)
    return data

In [6]:
[train_images, train_labels, test_images, test_labels] = load_pkl('../datasets/ASL_10_mDreal_224x224.pkl')
train_images = train_images.astype(np.float64)
test_images = test_images.astype(np.float64)
train_labels = np.argmax(train_labels, -1)
test_labels = np.argmax(test_labels, -1)

test_images = np.concatenate([train_images, test_images], 0)
test_labels = np.concatenate([train_labels, test_labels], 0)

# print(train_images.shape)
# print(train_labels.shape)
print(test_images.shape)
print(test_labels.shape)

(265, 3, 224, 224)
(265,)


In [7]:
# [train_images, train_labels, train_files,
#  test_images, test_labels, test_files] = load_pkl('../datasets/imit_ASL_10_mDreal.pkl')
# train_labels = np.argmax(train_labels, -1)
# test_labels = np.argmax(test_labels, -1)

# print(train_images.shape)
# print(train_labels.shape)
# print(test_images.shape)
# print(test_labels.shape)

## Dataloader

In [8]:
class CustomImageDataset(Dataset):
    def __init__(self, x, y):
        super().__init__()
        self.x = x
        self.y = y

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        temp_x = torch.tensor(self.x[idx], dtype=torch.float)
        temp_y = torch.tensor(self.y[idx], dtype=torch.long)
#         self.x = self.x[idx]
#         self.y = self.y[idx]
#         return self.x[idx], self.y[idx] 
        return temp_x, temp_y

## Classify mD

In [9]:
batch_size = 32
# training_data = CustomImageDataset(torch.from_numpy(train_images), torch.from_numpy(train_labels))
# train_loader = DataLoader(training_data, batch_size=batch_size, shuffle=False, collate_fn=lambda x: x)
testing_data = CustomImageDataset(torch.from_numpy(test_images), torch.from_numpy(test_labels))
test_loader = DataLoader(testing_data, batch_size=batch_size, shuffle=False, collate_fn=lambda x: x)

In [10]:
testing_features, testing_labels = next(iter(test_loader))
print(f"Feature batch shape: {testing_features.size()}")
print(f"Labels batch shape: {testing_labels.size()}")

Feature batch shape: torch.Size([32, 3, 224, 224])
Labels batch shape: torch.Size([32])


In [11]:
# iterate through the dataset:
for i, batch in enumerate(test_loader):
    print(f'{i}', '. batch length:', f'{len(batch[0])}')

0 . batch length: 32
1 . batch length: 32
2 . batch length: 32
3 . batch length: 32
4 . batch length: 32
5 . batch length: 32
6 . batch length: 32
7 . batch length: 32
8 . batch length: 9


In [12]:
def test(model):
    
    # test
    all_preds = []
    test_acc = []
    for data in test_loader:
        x, labels = data
        x = x.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        out = model(x)
        loss = criterion(out, labels)
        preds = np.argmax(out.cpu().detach().numpy(), -1)
        labels = labels.cpu().detach().numpy()
        acc = np.sum(preds == labels) / len(labels) * 100
        test_acc.append(acc)
        all_preds.append(preds)
    best_preds = [i for a in all_preds for i in a]
    acc = np.sum([1 for i in range(len(labels)) if labels[i] == best_preds[i]]) / len(labels) * 100
           
    return best_preds, acc

In [13]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = torchvision.models.vgg16(pretrained=False).to(device)
model.fc = torch.nn.Sequential(
    nn.Linear(
        in_features=2048,
        out_features=10
    ),
    nn.Softmax()
)
model.load_state_dict(torch.load('../models/model_vgg16_withTL_mDreal_imit_acc94.95.pth'))
model.eval()
# summary(model, (3, 128, 128))
print('device:', device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [ ]:
best_preds, acc = test(model)
acc